# NFT bitsCrunch challenge

### Objective
- to find trending nft collections on twitter and so sentiment Analysis on them

### Method followed
- This notebook collects tweets and save it in csv
- twint and tweepy was giving error even after following instructions
- hence i used seleni to scrape data
- Since selenium takes much tie to collect tweets so i collected ~1000 tweets.

### Machine setup
- chromedriver is needed as i have used chrome
#### how to get chromedriver
- open chrome and go to settings
- check chrome version and keep a note of it
- go to download chrome driver
- and download the same chrome driver as ou have noted down
- it will be downloaded as a zip folder.
- extract it and place it where ever you want but remember thw path to it
- then copy that path ttached with file name, and replace it in the code
- driver = webdriver.Chrome("path_to_folder/chromedriver")
- E.g :- driver = webdriver.Chrome("D:/chromedriver")
----------------------------------------------------------------------------------
- put your twitter username in place of username in below code
- then for password it will ask the input from user, give your password there

#### if you get an error of NoSuchElement
- then just keep re-running because sometimes it page doesn't get loaded and code get executd, so it needs proper loading time
- if error still persist then there might be some problem
- if you find difficulty then you can ignore my submission
- <b>time.sleep()</b> is used to tackle this exception

### Put it all together

In [ ]:
import csv
from getpass import getpass
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC

def get_tweet_data(card):
    try:
        name = card.find_element_by_xpath(".//span").text        
        twt_handle = card.find_element_by_xpath(".//span[contains(text(),'@')]").text
    #     twt = card.find_element_by_xpath("./div[2]/div[1]//span").text        
        tweet_body_response=card.find_elements_by_xpath("./div[2]/div/div[@class='css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0']//span")
        tweet_body=[]
        for i in tweet_body_response:
            tweet_body.append(i.text)
        tweet_body=" ".join(tweet_body)

        post_date = card.find_element_by_xpath(".//time").get_attribute("datetime")
        reply_cnt = card.find_element_by_xpath(".//div[@data-testid='reply']").text
        rt_cnt = card.find_element_by_xpath(".//div[@data-testid='retweet']").text
        like_cnt = card.find_element_by_xpath(".//div[@data-testid='like']").text
    except NoSuchElementException:
        return " "
        
    tweet = [name,twt_handle,tweet_body,post_date, reply_cnt, rt_cnt, like_cnt]
    
    return tweet
    
#create instance of webdriver
driver = webdriver.Chrome("/home/curaj/Pictures/chromedriver")

#go to twitter page
driver.get("https://twitter.com")
time.sleep(3)

#NAVIGATE TO LOGIN page
try:
    driver.find_element_by_xpath("//*[text()='Log in']").click()
except:
    driver.find_element_by_xpath("//*[text()='Sign in']").click()

time.sleep(3)

#enter user name
driver.find_element_by_xpath("//input[@name='text']").send_keys("username")
driver.find_element_by_xpath("//*[text()='Next']").click()
time.sleep(3)

#enter password
password=getpass() #get password
driver.find_element_by_xpath("//input[@name='password']").send_keys(password)
driver.find_element_by_xpath("//input[@name='password']").send_keys(Keys.RETURN)
time.sleep(7)

#finding the keyword
driver.find_element_by_xpath("//a[@aria-label='Search and explore']").click()
driver.find_element_by_xpath("//input[@aria-label='Search query']").click()

#collection of popular nfts
pop_nfts = ["Azuki", "MURI by Haus", "CryptoPunks","CLONE X - X TAKASHI MURAKAMI","(B)APETAVERSE","World of Women","Doodles","RTFKT - MNLTH"]

#looping over all nfts
for i in pop_nfts:
    driver.find_element_by_xpath("//input[@aria-label='Search query']").click()
    driver.find_element_by_xpath("//input[@type='text']").send_keys(i)
    driver.find_element_by_xpath("//input[@type='text']").send_keys(Keys.RETURN)
    time.sleep(8)

    #navigating to latest page
    driver.find_element_by_xpath("//*[text()='Latest']").click()
    time.sleep(8)

    #get all tweets on the page
    data = []
    
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling=True

    while scrolling:
        page_cards = driver.find_elements_by_xpath("//div[@class='css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu']")
#         print(len(page_cards))
        for card in page_cards:
            tweet = get_tweet_data(card)
            print(tweet)
            data.append(tweet)
#             if tweet:
#                 tweet_id = ''.join(tweet)
#                 if tweet_id not in tweet_ids:
#                     tweet_ids.add(tweet_id)
#                     data.append(tweet)
        #collection 1000 tweets for each nfts
        if len(data) >= 1000:
            scrolling=False
        scroll_attempt = 0
        while True:
            #check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(5)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                #end of scroll region 
                if scroll_attempt >= 3:
                    scrolling = False
                    break

                else:
                    time.sleep(5)
            else:
                last_position = curr_position
                break
                
    driver.find_element_by_xpath("//input[@aria-label='Search query']").click()
    for _ in i:
        driver.find_element_by_xpath("//input[@aria-label='Search query']").send_keys(Keys.BACKSPACE)
        
    df = pd.DataFrame(data,columns=['name','twt_handle','tweet_body',
        'post_date', 'reply_cnt', 'rt_cnt', 'like_cnt'])
    df.to_csv(i+'.csv',index=False)
                
    

········
['Hutsmark', '@Hutsmark22', 'Fantastic vault getting close to paying out:\n3 BAYC\n2 MAYC\n9 Coolcats\n1 Doodle\nWhat will be the next purchase??', '2022-03-28T11:05:01.000Z', '', '', '']
['PAPI CHEMA.', '@F1REGG', 'a mi me has amenazado varias veces', '2022-03-28T11:04:45.000Z', '', '', '']
['ちゃむ', '@chamu__u', '', '2022-03-28T11:04:35.000Z', '', '', '1']
['猫', '@nanasegnrou1', '', '2022-03-28T11:04:34.000Z', '', '', '']
['水鳴 倫紅', '@linc_for_music', '', '2022-03-28T11:04:16.000Z', '', '', '']
['WidowMaker', '@WidowMak3r', 'Quiero decir que la manera de hacer las cosas no está consiguiendo mandar un buen mensaje, el mensaje que estamos usando para acusar al machismo que existe ahora mismo solo genera odio entre géneros, cuando lo que hace falta es que todos cooperemos para erradicar el machismo', '2022-03-28T11:04:05.000Z', '', '', '']
['Coorgertln.eth', '@coorgertln', 'Woah life changing', '2022-03-28T11:03:53.000Z', '', '', '']
['BlockChainRobbie', '@chain_robbie', '@AzukiZe

['ふう', '@Q_07_2', '', '2022-03-28T10:49:45.000Z', '', '', '1']
['Elmond Golito', '@ElTenacitas', 'Gracias Miguel', '2022-03-28T10:49:35.000Z', '1', '', '2']
['茶々丸', '@tyatya_maru4646', '', '2022-03-28T10:49:28.000Z', '', '', '1']
['NASEER ALOBAIDI', '@alobaidi_naseer', '@AzukiZen', '2022-03-28T10:49:07.000Z', '', '', '']
['鯖野味噌煮', '@tacca_sabano', '', '2022-03-28T10:48:48.000Z', '1', '', '1']
['とと', '@toto__0912', '', '2022-03-28T10:47:52.000Z', '1', '', '1']
['eri*', '@eri_hani7', '', '2022-03-28T10:47:44.000Z', '', '', '2']
['うさぎり', '@usagiri_tsuki', '', '2022-03-28T10:47:17.000Z', '', '', '']
['Ignasi Abad', '@UpperGG_', 'Buen meme', '2022-03-28T10:46:45.000Z', '1', '', '']
['ふう', '@Q_07_2', '', '2022-03-28T10:46:35.000Z', '1', '', '1']
['ev0x.eth', '@ev0xeth', 'welcome to the garden! ', '2022-03-28T10:46:32.000Z', '', '', '']
['NSX-RRRRR', '@nsxrrrrr', '', '2022-03-28T10:46:22.000Z', '', '', '']
['要通販は固定ツイから', '@kanameEX', '', '2022-03-28T10:46:03.000Z', '1', '', '1']
['The Azuki B

['NFT Watcher', '@nftwatcher9487', '[3/5]  #EveraiDuo (0x9a38dec0590abc8c883d72e52391090e948ddf12)\n  - 3  AZUKI  (Buy:11, Sell:3)\n  - 3 CLONEX (Buy:7, Sell:0)\n  - 1 COOL (Buy:1, Sell:0)', '2022-03-28T10:29:42.000Z', '1', '', '']
['Blue Bunny Ape | tojear.eth', '@to_jear', 'Nice. It’s like an accessible  Azuki  then', '2022-03-28T10:29:40.000Z', '', '', '2']
['Yoshitaka 亀山 ᶦᵏᶻ', '@jdcomm_us', '', '2022-03-28T10:29:38.000Z', '', '', '1']
['ハマチカマ', '@7jhivKZi5iyjEqj', '', '2022-03-28T10:29:24.000Z', '', '', '1']
['Premala.eth', '@Premala_BRIC', '@HollyWoodNFTs  you need to attend on Weds!', '2022-03-28T10:29:10.000Z', '', '', '']
['jpops', '@jpops_eth', 'Day 1:\n\nWoke up, bought a “The Art Of Seasons” NFT because I thought the art looked sick. Definitely some kind of potential here imo especially with the  azuki  style art craze.\n\nBesides that I know admittedly quite little here. Cost me .215 altogether with gas, that’s at floor.', '2022-03-28T10:29:07.000Z', '1', '1', '4']
['𝖘𝖊𝖎𝖘𝖍𝖎

['100％！アピールちゃん【公式】', '@appealchan', '', '2022-03-28T10:08:08.000Z', '', '', '']
['ブラックサンダーさん【公式】', '@Black_Thunder_', '', '2022-03-28T10:08:04.000Z', '', '', '']
['Skarla', '@nicosdope', 'In the view of this week lol normal BAYC lands and hoodies,  Azuki  announcement and the best MNLTH final quest ! \n\nReady in the same time as crypto pump lol', '2022-03-28T10:07:31.000Z', '', '', '1']
['凪', '@nagitaishi', '', '2022-03-28T10:07:05.000Z', '', '', '']
['MBS毎日放送', '@MBS_fan', '', '2022-03-28T10:06:48.000Z', '', '', '']
['Jstpansy.sol', '@Jstpansy', "Don't miss out on this amazing crew and community. Solid support of alot of  azuki  members, if you missed out on the bean, don't snooze on  @TheEverai  been with these guys from the start.... amazing. LFG !! #ETHNFT   #NFTs   #Everai   #animation   #NFTCommunity   #NFTLaunch   #NFTartwork   #NFT", '2022-03-28T10:06:44.000Z', '2', '9', '13']
['Tomoko♩♬♡•*¨*•.¸¸♬︎', '@TomokoV6', '', '2022-03-28T10:06:42.000Z', '1', '', '1']
['Crypthomas', '@t

['0x中本とうる夜に駆ける.eth (, )', '@satoshi_trump', '', '2022-03-28T09:50:25.000Z', '', '2', '1']
['みずなの^ ^', '@omizumizu7', '', '2022-03-28T09:50:23.000Z', '1', '', '1']
['さらに、∞に幸せな５月生まれの蜃気楼(*´▽｀*)金銭的にも幸せになります', '@VTJRhLIiHrgznyi', '', '2022-03-28T09:49:49.000Z', '', '', '']
['甘々ジングク', '@ama_ama_loveban', '', '2022-03-28T09:49:39.000Z', '', '', '']
['nickay', '@azukigrinder', 'eth pumping just makes my road to get my first skater get harder and harder', '2022-03-28T09:49:25.000Z', '', '', '']
['Lunat1k', '@lunat1k33', 'current bags:   Azuki : 12x\nMuri: 8x\nBAYC: 11x\nMAYC: 20x\nDoodles: 3x\nNFTWorlds: 1x\nmfers: 4x\nMetroverse: 6x\nMavia: 2x\nKaiju: 23x\nCoolmans: 9x\nKarafuru: 12x\nSwampverse: 19x\nAlien Frens: 2x\nHabibiz: 10x\nEtherOrcs: 6x\nCrypto Coven: 5x', '2022-03-28T09:48:33.000Z', '', '', '1']
['kuruto＠仮想ニコ生主？&ミクチャ公認ライバー', '@kuruto_officer', '', '2022-03-28T09:47:28.000Z', '2', '', '']
['azuki', '@azuki09outi', '', '2022-03-28T09:47:21.000Z', '', '', '1']
['NFT Alerts by Moby.gg', 

...

['Chadu', '@Chadudubirimedu', '@ThePunksan  :)\n\ncool art + a manga story about  cryptopunks  created with the community. \n\nyou want more?', '2022-03-27T11:00:41.000Z', '', '', '1']
['Austin.eth', '@Cryptosamarai', 'Delisting your NFT makes the price up go. If you have any NFT listed you are part of the problem why the floor price has not been flipping for massive growth. #Ethereum   #Avalanche   #NEAR   #LUNA   #Solana   #MATIC   #BAYC   #cryptocurrency   #cryptopunks', '2022-03-27T10:56:57.000Z', '2', '4', '8']
 
 
['CryptoPunkV5', '@V5Punk', 'New CryptoPunk has been Listed On OpenSea and Raribe.Check out these links:\nRarible: https:// 24B823b10579895cCBE442d41b9Abd96Ed:70597905139945799125707368543265853523093781247924533613700238599009154891783 … OpenSea: https:// x2953399124f0cbb46d2cbacd8a89cf0599974963/70597905139945799125707368543265853523093781247924533613700238614402317680641/ …  #nft   #cryptopunks    #opensea   #rarible', '2022-03-27T10:56:09.000Z', '1', '2', '3']
['Mar

['NTC.eth', '@nftradersclub', 'CLONE   X  -  X   TAKASHI MURAKAMI   #NFT  holders bought\n\n17 pcs   @MuriNFT 13 pcs   @ETHTerrestrials   ( minted )\n10 pcs   @TheEverai  Heroes: Duo  ( minted )\n\nin last 1 hours  #NFTs   #NFTCommunity', '2022-03-27T16:27:30.000Z', '', '10', '11']
['Trading Benjamins', '@tradinbenjamins', 'When is  @nike  and  @RTFKTstudios  going to give  @naomiosaka  a   @takashipom   designed  Clone   X Can’t think of a more fitting person to rep the  Clone  brand', '2022-03-27T16:11:41.000Z', '1', '', '2']
['NFT FLOOR PRICE UPDATES', '@0xbotfather', 'Featured Collection:  @TheLoomSociety Daily floor price update\n\nCryptopunks: 70.95Ξ (  -1.32%)\nGoons of Balatroon: 0.17Ξ (  3.03%)\nBored Ape YC: 103.69Ξ (  0.67%)\nMutant Ape Yacht Club: 22.69Ξ (  -1.35%) CLONE   X    X   TAKASHI MURAKAMI : 15.0Ξ (  0.13%)', '2022-03-27T16:06:58.000Z', '1', '', '']
['Trengible - Discover the nft trends', '@Trengible', ' - Ccw_RRMaxi just bought CloneX #7366 for 16.5 ETH ($51,843.0

['NFT FLOOR PRICE UPDATES', '@0xbotfather', 'Featured Collection:  @TheLoomSociety Daily floor price update\n\nCryptopunks: 72.69Ξ (  -0.27%)\nGoons of Balatroon: 0.149Ξ (  -5.7%)\nBored Ape YC: 103.0Ξ (  0.0%)\nMutant Ape Yacht Club: 23.0Ξ (  -0.86%) CLONE   X    X   TAKASHI MURAKAMI : 14.98Ξ (  1.35%)', '2022-03-26T08:07:01.000Z', '1', '', '']
['Trengible - Discover the nft trends', '@Trengible', ' - nftcrypto3 just bought CloneX #1895 for 32 ETH ($99,887.04)\n\nCollection:  CLONE   X  -  X   TAKASHI MURAKAMI Date: 2022-03-26 07:09:02 (GMT 0)\n\nDetail:  https:// 1895-0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b-16311 …', '2022-03-26T07:10:19.000Z', '', '', '']
['NFT Tracker SOLE Tools', '@soletoolsapp', 'More here:  https:// Ξ69.42   CLONE   X  -  X   TAKASHI MURAKAMI  (#13980)\nΞ24     Mutant Ape Yacht Club (#2564)\nΞ23.99  Azuki (#8808) #NFTCollection   #BlockchainGaming   #Bittrex   #Poland', '2022-03-26T07:09:02.000Z', '', '', '2']
['NFT Whale Alert', '@nftwhalealert', 'Hourly Top

['NFT FLOOR PRICE UPDATES', '@0xbotfather', 'Featured Collection:  @TheLoomSociety Daily floor price update\n\nCryptopunks: 72.89Ξ (  1.31%)\nBored Ape YC: 103.0Ξ (  0.0%)\nMutant Ape Yacht Club: 22.5Ξ (  2.65%) CLONE   X    X   TAKASHI MURAKAMI : 13.9Ξ (  0.0%)\nVeeFriends: 15.4Ξ (  4.41%)', '2022-03-24T22:06:59.000Z', '1', '', '1']
['Trengible - Discover the nft trends', '@Trengible', ' - Aych_Nano just bought CloneX #8994 for 17.5 ETH ($54,300.75)\n\nCollection:  CLONE   X  -  X   TAKASHI MURAKAMI Date: 2022-03-24 20:52:32 (GMT 0)\n\nDetail:  https:// 8994-0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b-13554 …', '2022-03-24T20:54:16.000Z', '', '', '1']
['NftBossAlert', '@nftbossalert', 'Collection:  CLONE   X  -  X   TAKASHI MURAKAMI CloneX #6479 was bought\nPrice: 15.5 ETH ($48105.8) \nSeller: ArtGuyHere\nBuyer: TwDDDD-nft https:// f5d44e70224e2e23fdcdd2c053f30ada28b/12201 …  #CloneX   #NFTSales   #NFTWhale', '2022-03-24T17:46:04.000Z', '', '', '']
['nft watcher bot', '@nft_watcher_bot

['NFT Alerts', '@nftflooralerts', " Top Whale SELLS  (Last hour):\n\n(B)APETAVERSE: 3 (avg: 0.58 Ξ) CLONE   X  -  X   TAKASHI MURAKAMI : 2 (avg: 27.34 Ξ)\nFreaks N' Guilds - OnChain: 2 (avg: 0.05 Ξ)\nGold Hunt Game: 2 (avg: 0.01 Ξ)\n0xe935e764da0f1a2597f9c1bf13a64515bff9f7d7: 1 (avg: 36.66 Ξ)", '2022-03-23T03:02:05.000Z', '', '', '']
['NFT Whale Alert', '@nftwhalealert', 'Hourly Top Volume (LooksRare wash trades included)\nTerraforms by Mathcastles: Ξ425.64 ($1,268,300.79)\n(B)APETAVERSE: Ξ269.96 ($804,417.66) CLONE   X  -  X   TAKASHI MURAKAMI : Ξ127.94 ($381,232.19)\nBored Ape Yacht Club: Ξ108.00 ($321,813.00)', '2022-03-23T03:00:10.000Z', '1', '2', '6']
['Trengible - Discover the nft trends', '@Trengible', ' - Rentaro just bought CloneX #10788 for 19.69 ETH ($58,229.63)\n\nCollection:  CLONE   X  -  X   TAKASHI MURAKAMI Date: 2022-03-23 02:56:27 (GMT 0)\n\nDetail:  https:// 10788-0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b-6801 …', '2022-03-23T02:57:15.000Z', '', '', '']
['nft watche